# Image overpass times
The image overpasss times are estimated using the SOIT utility. It will take some work to extract the information and put it in order into the validation datatable.

In [140]:
import pandas as pd
df = pd.read_csv('../data/validation_dataset/validation_dataset.csv')
df.head()

,case_number,region,start_date,center_lon,center_lat,center_x,center_y,month,sea_ice_fraction,mean_sea_ice_concentration,...,artifacts,qa_analyst,qa_reviewer,fl_analyst,fl_reviewer,psd_file,floe_obscuration,landfast_obscuration,modis_cloud_error,notes
0,1,baffin_bay,2022-09-11,-91.527525,77.801013,-962500,-912500,9,1.0,0.573,...,yes,daniel,emma,daniel,NaN,yes,heavy,NaN,no,reclassified landfast ice (likely cloud deck)
1,1,baffin_bay,2022-09-11,-91.527525,77.801013,-962500,-912500,9,1.0,0.573,...,yes,ethan,daniel,daniel,NaN,yes,heavy,NaN,no,reclassified landfast ice (likely cloud deck)
2,2,baffin_bay,2015-03-12,-81.964289,76.057935,-912500,-1212500,3,1.0,0.858,...,no,daniel,emma,NaN,NaN,NaN,NaN,NaN,no,NaN
3,2,baffin_bay,2015-03-12,-81.964289,76.057935,-912500,-1212500,3,1.0,0.858,...,no,ethan,daniel,NaN,NaN,NaN,NaN,NaN,no,NaN
4,3,baffin_bay,2012-04-19,-79.579288,75.637235,-887500,-1287500,4,1.0,0.873,...,yes,daniel,emma,NaN,NaN,NaN,NaN,heavy,no,NaN


In [132]:
# I'm running this multiple times and finding that it sometimes fails and sometimes doesn't.
# On each region, it seems that it first fails a few times then succeeds. Is in an API lock?
# Perhaps this is an error in the API for skyfield? How can we check?
SPACEUSER="daniel_watkins@brown.edu"
SPACEPSWD="gajtox-fetmi4-junbuG"
for region in np.unique(df.region):
    for row, data in df.loc[(df.satellite=='terra') & (df.region == region)].iterrows():    
        print(data.case_number, data.region)
    
        soit_name = 'passtimes_lat{lat}_lon{lon}_{date1}_{date2}.csv'.format(
                        lat = data.center_lat,
                        lon = data.center_lon,
                        date1 = pd.to_datetime(data.start_date).strftime('%m%d%Y'),
                        date2 = (pd.to_datetime(data.start_date) + pd.to_timedelta('1d')).strftime('%m%d%Y'))
        if not os.path.exists('../data/metadata/soit/' + soit_name):
                
            startdate = data.start_date
            enddate = data.start_date # automatically adds 1 day either way
            lat = data.center_lat
            lon = data.center_lon
            savepath = '../data/metadata/soit'
            
            function_path = "/Users/dwatkin2/Documents/research/packages/ice-floe-tracker-pipeline/" + \
                        "satellite-overpass-identification-tool/satellite_overpass_identification_tool.py"
            ! python $function_path --SPACEUSER $SPACEUSER --SPACEPSWD $SPACEPSWD --startdate $startdate --enddate $enddate --centroid-lat $lat --centroid-lon $lon --csvoutpath $savepath

1 baffin_bay
2 baffin_bay
3 baffin_bay
4 baffin_bay
5 baffin_bay
6 baffin_bay
7 baffin_bay
8 baffin_bay
9 baffin_bay
10 baffin_bay
11 baffin_bay
12 baffin_bay
13 baffin_bay
14 baffin_bay
15 baffin_bay
16 baffin_bay
17 baffin_bay
18 baffin_bay
19 baffin_bay
20 baffin_bay
21 baffin_bay
22 barents_kara_seas
23 barents_kara_seas
Outpath ../data/metadata/soit
Timeframe starts on ['03', '04', '2019'], and ends on ['03', '04', '2019']
Coordinates (x, y): (76.34205355, 26.86981309)
Traceback (most recent call last):
  File "/Users/dwatkin2/Documents/research/packages/ice-floe-tracker-pipeline/satellite-overpass-identification-tool/satellite_overpass_identification_tool.py", line 433, in <module>
    main()
    ~~~~^^
  File "/Users/dwatkin2/Documents/research/packages/ice-floe-tracker-pipeline/satellite-overpass-identification-tool/satellite_overpass_identification_tool.py", line 429, in main
    get_passtimes(**vars(args))
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^
  File "/Users/dwatkin2/Documents/rese

In [144]:
df['pass_time'] = pd.NaT

for name, group in df.groupby('case_number'):
    data = group.iloc[0,:]
    soit_name = 'passtimes_lat{lat}_lon{lon}_{date1}_{date2}.csv'.format(
                    lat = data.center_lat,
                    lon = data.center_lon,
                    date1 = pd.to_datetime(data.start_date).strftime('%m%d%Y'),
                    date2 = (pd.to_datetime(data.start_date) + pd.to_timedelta('1d')).strftime('%m%d%Y'))
    if os.path.exists('../data/metadata/soit/' + soit_name):
        df_soit = pd.read_csv('../data/metadata/soit/' + soit_name).set_index('satellite')
        for idx, row_data in group.iterrows():
            satellite = row_data.satellite
            df.loc[idx, 'pass_time'] = pd.to_datetime(df_soit.loc[satellite, "overpass time"].replace('Z', ''))

In [145]:
df.loc[df.satellite=='terra', ['case_number', 'region', 'start_date', 'pass_time', 'center_lat', 'center_lon']].loc[df.pass_time.isnull()]

,case_number,region,start_date,pass_time,center_lat,center_lon
45,23,barents_kara_seas,2019-03-04,NaT,76.342054,26.869813
47,24,barents_kara_seas,2015-04-26,NaT,79.921873,31.122470
63,32,barents_kara_seas,2014-05-01,NaT,76.977679,60.875947
107,54,beaufort_sea,2015-05-16,NaT,70.444116,-136.674838
137,69,bering_chukchi_seas,2013-04-23,NaT,58.238610,-172.803765
141,71,bering_chukchi_seas,2009-05-23,NaT,61.824479,-171.179620
175,88,east_siberian_sea,2014-08-08,NaT,73.154624,-172.259659
177,89,east_siberian_sea,2014-05-11,NaT,76.035464,146.864391
195,98,east_siberian_sea,2019-06-15,NaT,76.193555,154.942385
211,106,greenland_sea,2014-09-11,NaT,78.793415,-13.431029


In [150]:
# Manual identification of overpass time from NASA Worldview orbit images
colnames = ['case_number', 'region', 'start_date', 'satellite', 'pass_time']
manual_identification = [
    [23,    'barents_kara_seas', 	'2019-03-04',    'aqua',    '2019-03-04 09:45'],
    [23,    'barents_kara_seas', 	'2019-03-04',    'terra',   '2019-03-04 11:03'],
    [24,    'barents_kara_seas', 	'2015-04-26',    'aqua',    '2015-04-26 08:09'],
    [24,    'barents_kara_seas', 	'2015-04-26',    'terra',   '2015-04-26 12:40'],
    [32,    'barents_kara_seas', 	'2014-05-01',    'aqua',    '2014-05-01 07:18'],
    [32,    'barents_kara_seas', 	'2014-05-01',    'terra',   '2014-05-01 08:35'],
    [54,    'beaufort_sea', 	        '2015-05-16',    'aqua',    '2015-05-16 20:52'],
    [54,    'beaufort_sea',	        '2015-05-16',    'terra',   '2015-05-16 20:34'],
    [69,    'bering_chukchi_seas', 	'2013-04-23',    'aqua',    '2013-04-23 00:21'],
    [69,    'bering_chukchi_seas', 	'2013-04-23',    'terra',   '2013-04-23 23:11'],
    [71,    'bering_chukchi_seas', 	'2009-05-23',    'aqua',    '2009-05-23 00:17'],
    [71,    'bering_chukchi_seas', 	'2009-05-23',    'terra',   '2009-05-23 23:05'],
    [88,    'east_siberian_sea', 	'2014-08-08',    'aqua',    '2014-08-08 22:38'],
    [88,    'east_siberian_sea', 	'2014-08-08',    'terra',   '2014-08-08 23:56'],
    [89,    'east_siberian_sea', 	'2014-05-11',    'aqua',    '2014-05-11 01:20'],
    [89,    'east_siberian_sea', 	'2014-05-11',    'terra',   '2014-05-11 02:37'],
    [98,    'east_siberian_sea', 	'2019-06-15',    'aqua',    '2019-06-15 01:37'],
    [98,    'east_siberian_sea', 	'2019-06-15',    'terra',   '2019-06-15 02:54'],
    [106,   'greenland_sea', 	    '2014-09-11',    'aqua',    '2014-09-11 10:55'],
    [106,   'greenland_sea', 	    '2014-09-11',    'terra',   '2014-09-11 13:49'],
    [143,   'hudson_bay', 	        '2015-07-20', 	  'aqua',   '2015-07-20 18:33'],
    [143,   'hudson_bay', 	        '2015-07-20', 	  'terra',  '2015-07-20 16:25'],
]
df_manual = pd.DataFrame(data=manual_identification, columns=colnames)
df_manual['pass_time'] = [pd.to_datetime(d) for d in df_manual['pass_time']]

In [147]:
df.loc[df.satellite=='terra'].groupby('region').count()[['case_number', 'pass_time']]

,case_number,pass_time
region,,
baffin_bay,21,21
barents_kara_seas,21,18
beaufort_sea,21,20
bering_chukchi_seas,21,19
east_siberian_sea,21,18
greenland_sea,21,20
hudson_bay,21,20
laptev_sea,21,21
sea_of_okhostk,21,21


In [159]:
for idx, data in df.loc[df.pass_time.isnull()].iterrows():
    manual_time = pd.to_datetime(df_manual.loc[(df_manual.case_number == data.case_number) & (df_manual.satellite == data.satellite), "pass_time"].values.squeeze())
    df.loc[idx, 'pass_time'] = manual_time


In [167]:
df.to_csv('../data/validation_dataset/validation_dataset.csv')

In [166]:
df_init.head()

,case_number,region,start_date,center_lon,center_lat,center_x,center_y,month,sea_ice_fraction,mean_sea_ice_concentration,...,artifacts,qa_analyst,qa_reviewer,fl_analyst,fl_reviewer,psd_file,floe_obscuration,landfast_obscuration,modis_cloud_error,notes
0,1,baffin_bay,2022-09-11,-91.527525,77.801013,-962500,-912500,9,1.0,0.573,...,yes,daniel,emma,daniel,NaN,yes,heavy,NaN,no,reclassified landfast ice (likely cloud deck)
1,1,baffin_bay,2022-09-11,-91.527525,77.801013,-962500,-912500,9,1.0,0.573,...,yes,ethan,daniel,daniel,NaN,yes,heavy,NaN,no,reclassified landfast ice (likely cloud deck)
2,2,baffin_bay,2015-03-12,-81.964289,76.057935,-912500,-1212500,3,1.0,0.858,...,no,daniel,emma,NaN,NaN,NaN,NaN,NaN,no,NaN
3,2,baffin_bay,2015-03-12,-81.964289,76.057935,-912500,-1212500,3,1.0,0.858,...,no,ethan,daniel,NaN,NaN,NaN,NaN,NaN,no,NaN
4,3,baffin_bay,2012-04-19,-79.579288,75.637235,-887500,-1287500,4,1.0,0.873,...,yes,daniel,emma,NaN,NaN,NaN,NaN,heavy,no,NaN
